## CL-Gym Example: Stable A-GEM on Rotated MNIST

In this example, we use Averaged Gradient Episodic Memory (A-GEM) to train on Rotated MNIST benchmark. We use the stable version of [AGEM](https://arxiv.org/abs/1812.00420.pdf) using [Stable SGD](https://proceedings.neurips.cc/paper/2020/file/518a38cc9a0173d0b2dc088166981cf8-Paper.pdf) parameters.

## 1. Defining Parameters
First, we need to define our parameters/config for our experiment.
We define all our parameters inside a python dictionary. The parameters define different aspects of continual learning examples. For example:
-  How many tasks should we learn?
-  What our batch-size will be?
-  What Optimizer will we use?
-  Where should we store our outputs?

In [1]:
import torch
import cl_gym as cl
# first let's create params/config for our experiment

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # benchmark
            'num_tasks': 5,
            'epochs_per_task': 5,
            'per_task_memory_examples': 25,
            'batch_size_train': 64,
            'batch_size_memory': 32,
            'batch_size_validation': 256,

            # algorithm
            'optimizer': 'SGD',
            'learning_rate': 0.01,
            'momentum': 0.8,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:2' if torch.cuda.is_available() else 'cpu'), }

    trial_id = str(uuid.uuid4())
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

## 2. Training our continual learning algorithm

Before seeing the code, let's explain the components one more time:
### 2.1 Benchmark
* We use the `RotatedMNIST` benchmark for this example. The benchmark includes gradual rotations of MNIST digits for each task. Something like this.
<div>
<img src="https://user-images.githubusercontent.com/8312051/122752221-845a0300-d245-11eb-8892-7c4119ffe1a5.png" width="300"/>
</div>

In CL-Gym we use RotatedMNIST as follows:
```python
benchmark = cl.benchmarks.RotatedMNIST(num_tasks=5)
```
---------

### 2.2 Backbone

* We use a MLP model with two hidden layers like this:
<div>
<img src="https://user-images.githubusercontent.com/8312051/122753641-67beca80-d247-11eb-87d3-dec5cc2e63d6.png" width="300"/>
</div>

To import our backbone, we use:
```python
backbone = cl.backbones.MLP2Layers(input_dim=784, hidden_dim_1=100, hidden_dim_2=100, output_dim=10)
```

You can also create your own PyTorch models. The backbone in CL-Gym is simply a lightweight wrapper around PyTorch's ``nn.Module``.

--------

### 2.3 Collecting metrics with Callbacks

The `MetricCollector` callback evaluates the model at the end of each epoch, logs the metrics, plots the accuraies to file, and stores the validation accuracies as numpy arrays to file (see outputs folder).
```python
metric_callback = cl.callbacks.MetricCollecto(num_tasks=5,
                                              eval_interval='epoch',
                                              epochs_per_task=1)
```

-------

### 2.4  Using off-the-shelf continual learning algorithms

CL-Gym includes several continual learning algorithms. Here we use A-GEM algorithm with better parameters than the original paper:

```python
cl.algorithms.AGEM(backbone, benchmark, params)
```

You can also use other algorithms. For example, for Experience Replay method, you can use:
```python
cl.algorithms.ERRingBuffer(backbone, benchmark, params)
```


-------

### 2.5 Gluing everything together with the Trainer

The `Trainer` will orchestrate the experiment by handling the non-research part of continual learning experiments.

```
trainer = cl.trainer.ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])
```


The code below implements this note:

In [2]:
def train(params):
    # benchmark: Rotated MNIST
    benchmark = cl.benchmarks.PermutedMNIST(num_tasks=params['num_tasks'],
                                           per_task_memory_examples=params['per_task_memory_examples'],)
    
    # backbone: MLP with 2 hidden layers
    backbone = cl.backbones.MLP2Layers(input_dim=784, hidden_dim_1=32, hidden_dim_2=32, output_dim=10)

    # Algorithm: A-GEM
    algorithm = cl.algorithms.AGEM(backbone, benchmark, params)
    # algorithm = cl.algorithms.ERRingBuffer(backbone, benchmark, params)

    # Callbacks
    metric_manager_callback = cl.callbacks.MetricCollector(num_tasks=params['num_tasks'],
                                                           eval_interval='epoch',
                                                           epochs_per_task=params['epochs_per_task'])

    # Make trainer
    trainer = cl.trainer.ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])

    trainer.run()
    print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
    print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())


params = make_params()
train(params)

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 92.92, 'loss': 0.000972979424893856}
[2] Eval metrics for task 1 >> {'accuracy': 94.87, 'loss': 0.000679773323982954}
[3] Eval metrics for task 1 >> {'accuracy': 95.72, 'loss': 0.0005617750629782677}
[4] Eval metrics for task 1 >> {'accuracy': 96.35, 'loss': 0.0004930462460964918}
[5] Eval metrics for task 1 >> {'accuracy': 96.32, 'loss': 0.00048549841716885567}
---------------------------- Task 2 -----------------------
[6] Eval metrics for task 1 >> {'accuracy': 81.74, 'loss': 0.0023741892993450163}
[6] Eval metrics for task 2 >> {'accuracy': 94.54, 'loss': 0.0007384905070066452}
[7] Eval metrics for task 1 >> {'accuracy': 73.25, 'loss': 0.004258131301403046}
[7] Eval metrics for task 2 >> {'accuracy': 95.89, 'loss': 0.0005414123661816121}
[8] Eval metrics for task 1 >> {'accuracy': 74.08, 'loss': 0.00424471988081932}
[8] Eval metrics for task 2 >> {'accuracy': 96.36, 'loss': 0.000

In [5]:
ref

[25] Eval metrics for task 1 >> {'accuracy': 57.84, 'loss': 0.009022631740570069}
[25] Eval metrics for task 2 >> {'accuracy': 74.56, 'loss': 0.004014899867773056}
[25] Eval metrics for task 3 >> {'accuracy': 83.45, 'loss': 0.002212368157505989}
[25] Eval metrics for task 4 >> {'accuracy': 89.72, 'loss': 0.0013752082452178002}
[25] Eval metrics for task 5 >> {'accuracy': 96.04, 'loss': 0.000493720968067646}
final avg-acc 80.32200000000002
final avg-forget 19.782499999999995

batch
[25] Eval metrics for task 1 >> {'accuracy': 24.49, 'loss': 0.019291391348838807}
[25] Eval metrics for task 2 >> {'accuracy': 37.36, 'loss': 0.01072963581085205}
[25] Eval metrics for task 3 >> {'accuracy': 49.19, 'loss': 0.008518594717979432}
[25] Eval metrics for task 4 >> {'accuracy': 74.51, 'loss': 0.0034106424331665037}
[25] Eval metrics for task 5 >> {'accuracy': 96.72, 'loss': 0.00043869449868798256}
final avg-acc 56.45399999999999
final avg-forget 49.925


SyntaxError: invalid syntax (461557611.py, line 2)